In [ ]:
'''
Date: 2024-10-07
Last Edit Date: 2024-11-15
Description: This notebook can be run to ensure that the values in redis have the same valid_from_date as those in the underlying table,
i.e. that the Redis has been updated appropriately. 
'''
import time
from datetime import timedelta
import pickle
import redis
import os
import pandas as pd


from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/user/ficc/creds.json"
bq_client = bigquery.Client()
project = "eng-reactor-287421"

BQ_CLIENT = bigquery.Client()
job_config = bigquery.job.QueryJobConfig()

REFERENCE_DATA_REDIS_CLIENT = redis.Redis(host='10.14.140.37', port=6379, db=0)


def sqltodf(sql, limit=''):
    if limit != '': limit = f' WHERE trade_date < DATE({limit})'
    bqr = BQ_CLIENT.query(sql + limit).result()
    return bqr.to_dataframe()

In [ ]:
#df = pd.read_parquet('/home/user/ficc/file.parquet')
df =sqltodf('select cusip, ref_valid_from_date from `auxiliary_views.trade_history_latest_ref_data_minimal_exclusions`')
len(df)

# Loop through the DataFrame rows and compare the dates
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing CUSIPs"):
    cusip = row['cusip']
    csv_ref_valid_from_date = pd.to_datetime(row['ref_valid_from_date'])  # Convert CSV date to datetime
    
    # Retrieve the data from Redis (assuming the Redis client is properly configured)
    redis_data = REFERENCE_DATA_REDIS_CLIENT.get(cusip)

    # Unpickle the data if it's not None
    if redis_data:
        redis_data_unpickled = pickle.loads(redis_data)
        
        # Check if the 'ref_valid_from_date' is in the data from Redis
        redis_ref_valid_from_date = redis_data_unpickled.get('ref_valid_from_date')
        
        # Compare the dates if the field exists in Redis data
        if redis_ref_valid_from_date:
            redis_ref_valid_from_date = pd.to_datetime(redis_ref_valid_from_date)  # Convert Redis date to datetime 

            # Check if the dates match, if not print or log the details
            if csv_ref_valid_from_date != redis_ref_valid_from_date:
                print(f"Mismatch for CUSIP {cusip}: CSV date {csv_ref_valid_from_date}, Redis date {redis_ref_valid_from_date}")
        else:
            print(f"Missing 'ref_valid_from_date' for CUSIP {cusip} in Redis.")
    else:
        print(f"No Redis entry found for CUSIP {cusip}.")
